# K-Nearest Neighbors

## Data Preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
dataset_train = pd.read_csv("../train.csv")
X_train = dataset_train.drop(["Survived"], axis=1)
y_train = dataset_train["Survived"]
X_test = pd.read_csv("../test.csv")


In [3]:
numeric_features = ["Age", "SibSp", "Parch", "Fare"]
categorical_features = ["Sex", "Pclass", "Embarked"]


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)


In [5]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)


In [6]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


## Define the model

In [7]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()


## Create the complete pipeline

In [8]:
pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", classifier)])


## Define hyperparameter grid for Grid Search

In [9]:
param_grid = {
    "classifier__n_neighbors": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
    ],
    "classifier__weights": ["uniform", "distance"],
    "classifier__metric": ["euclidean", "manhattan"],
}

## Grid search with 10-fold cross-validation

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipeline, param_grid, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print("Best parameters from Grid Search:", grid_search.best_params_)
print("Best score from Grid Search:", grid_search.best_score_)


Fitting 10 folds for each of 40 candidates, totalling 400 fits


Best parameters from Grid Search: {'classifier__metric': 'manhattan', 'classifier__n_neighbors': 10, 'classifier__weights': 'uniform'}
Best score from Grid Search: 0.8159550561797753


## Evaluate the model using k-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=10)
print("Cross-validation scores:", cross_val_scores)
print("Average cross-validation score:", np.mean(cross_val_scores))


Cross-validation scores: [0.8        0.83146067 0.76404494 0.83146067 0.86516854 0.79775281
 0.83146067 0.78651685 0.82022472 0.83146067]
Average cross-validation score: 0.8159550561797753


## Make predictions using the best model from Grid Search

In [12]:
y_pred = grid_search.best_estimator_.predict(X_test)


In [13]:
print(y_pred)


[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 1 1 1 1 0 0 1 0 0 0]


## Export to csv

In [14]:
results = pd.DataFrame({"PassengerId": X_test["PassengerId"], "Survived": y_pred})
results.to_csv("knn_submission.csv", index=False)
